## Author: Jesús Salazar Gómez, Costa Rica
## Course: Applied Data Science Capstone, week 4 and 5 (project), Coursera

## LinkedIn: https://www.linkedin.com/in/jesagocr/
## GitHub: https://github.com/jesagocr

In [5]:
# to open the url
from urllib.request import urlopen

# to work with the api
import json
import requests
import pandas as pd

# to work with the geolocalization data
!pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

You should consider upgrading via the 'c:\users\jesag\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\jesag\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.
Libraries imported.
